In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ichha\\OneDrive\\Desktop\\main-end-to-end-text-summarization\\research'

In [3]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\ichha\\OneDrive\\Desktop\\main-end-to-end-text-summarization'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    logging_steps: int
    save_strategy: str
    remove_unused_columns: bool
    report_to: str
    gradient_accumulation_steps: int
    dataloader_num_workers: int
    fp16: bool
    seed: int
    load_best_model_at_end: bool
    metric_for_best_model: str
    

In [13]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            logging_steps = params.logging_steps,
            save_strategy = params.save_strategy,
            remove_unused_columns= params.remove_unused_columns,
            report_to= params.report_to,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            dataloader_num_workers= params.dataloader_num_workers,
            fp16= params.fp16,
            seed= params.seed,
            load_best_model_at_end= params.load_best_model_at_end,
            metric_for_best_model= params.metric_for_best_model
        )

        return model_trainer_config

In [15]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [16]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
        output_dir=self.config.root_dir,
        per_device_train_batch_size=1,per_device_eval_batch_size=1,logging_steps=500,save_strategy='no',remove_unused_columns=True,report_to='none',
        gradient_accumulation_steps=8,dataloader_num_workers=0,fp16=False,seed=42,load_best_model_at_end=True, metric_for_best_model="loss")

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"t5-small"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-23 17:20:39,428: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-23 17:20:39,434: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-23 17:20:39,436: INFO: common: created directory at: artifacts]
[2025-04-23 17:20:39,438: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\ichha\anaconda3\envs\textS\envs\mainS\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ichha\.cache\huggingface\hub\models--google-t5--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regula

[2025-04-23 17:20:41,260: WARNING: file_download: Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`]


C:\Users\ichha\AppData\Local\Temp\ipykernel_33960\525451495.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
100%|██████████| 306/306 [21:47<00:00,  4.27s/it]


{'train_runtime': 1307.2333, 'train_samples_per_second': 1.88, 'train_steps_per_second': 0.234, 'train_loss': 2.254049064287173, 'epoch': 2.99}
